# LangGraph practice # 3



## 구성도

In [1]:
import os,sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('utils'), '..')))
from module.utils import * 
from module.prompt import * 
from module.custom_model import *

In [2]:
from typing import TypedDict, Annotated, List, Literal
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage,HumanMessage

In [3]:
strt_langsmith('practice_3')

LangSmith 추적을 시작합니다.
[프로젝트명]
practice_3


In [4]:

class State(TypedDict):
    messages : Annotated[list,add_messages] # 상호 대화

In [5]:
def _swap_roles(messages):
    # 메시지의 역할을 교환: 시뮬레이션 사용자 단계에서 메시지 타입을 AI -> Human, Human -> AI 로 교환합니다.
    new_messages = []
    for m in messages:
        if isinstance(m, AIMessage):
            # AIMessage 인 경우, HumanMessage 로 변환합니다.
            new_messages.append(HumanMessage(content=m.content))
        else:
            # HumanMessage 인 경우, AIMessage 로 변환합니다.
            new_messages.append(AIMessage(content=m.content))
    return new_messages

def ai_response(messages)->list:
    prompt = get_prompt_assistant()
    llm = get_gemini()
    chain = prompt | llm | StrOutputParser()
    result = chain.invoke({'messages':messages})
    return result

def persona_response(messages)->list:
    swap_messages = _swap_roles(messages)
    prompt = get_prompt_persona()
    llm = get_gemini()
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({'messages':swap_messages})

In [8]:
def ai_counselor(state : State) -> State:
    """
        고객의 요청을 응답해주는 상담 챗봇
    """
    messages = state['messages']
    response = ai_response(messages)
    return State({'messages':[AIMessage(content=response)]})


def persona_guest(state : State) -> State:
    """
        컴플레인 요청을 하는 손님 역할 
    """
    messages = state['messages']
    response = persona_response(messages)
    return State({'messages':[HumanMessage(content=response)]})

def should_coninue(state:State)-> Literal['ai_counselor',END]:
    messages=state['messages']
    latest_messages = messages[-1].content
    if len(messages) > 10:
        return END
    if latest_messages == 'FINISHED':
        return END
    else:
        return 'ai_counselor'


In [9]:
state_graph = StateGraph(State)
state_graph.add_node('ai_counselor',ai_counselor)
state_graph.add_node('persona_guest',persona_guest)

state_graph.add_edge(START,'ai_counselor')
state_graph.add_edge('ai_counselor','persona_guest')
state_graph.add_conditional_edges(
    source='persona_guest',
    path=should_coninue
)
memory = get_check_pointer()
graph = state_graph.compile(checkpointer=memory)

In [27]:
# visualize_graph(graph)
# print(graph.get_graph().draw_mermaid())

In [ ]:
uuid = get_random_uuid()
config = get_runnable_config(recursion_limit=10,thread_id=uuid)

query = '작년에 발생한 지연으로인한 피해보상요구한다'
inputs = State({'messages':[HumanMessage(content=query)]})
# for event in graph.stream(inputs,config):
#     print(event)
event = graph.invoke(inputs,config)
print(event)
# invoke_graph(graph,inputs,config)
# stream_graph(graph,inputs,config)

{'messages': [HumanMessage(content='작년에 발생한 지연으로인한 피해보상요구한다', additional_kwargs={}, response_metadata={}, id='d92e5803-8626-46c6-b92f-5ad366a91187'), AIMessage(content='고객님, 작년에 발생한 지연으로 인해 불편을 겪으신 점 진심으로 사과드립니다.\n\n피해 보상 요구에 대해 안내해 드리겠습니다.\n\n먼저, 정확한 보상 처리를 위해 몇 가지 정보가 필요합니다. 번거로우시겠지만 아래 내용을 확인하시어 회신 부탁드립니다.\n\n1.  **예약 번호 또는 항공권 번호:**\n2.  **탑승하신 항공편 정보 (출발지, 도착지, 날짜, 항공편명):**\n3.  **지연 발생 당시 상황에 대한 간략한 설명 (예: 지연 시간, 지연 사유 등):**\n4.  **요청하시는 보상의 종류 (예: 환불, 마일리지 적립, 할인 쿠폰 등):**\n5.  **연락처 (전화번호, 이메일 주소):**\n\n위 정보를 보내주시면, 저희 보상팀에서 해당 내용을 검토하여 신속하게 처리해 드리겠습니다.\n\n보상 규정 및 절차에 따라 처리되며, 경우에 따라 추가적인 서류 제출을 요청드릴 수도 있습니다.\n\n다시 한번 불편을 드린 점 사과드리며, 고객님의 이해와 협조에 감사드립니다.', additional_kwargs={}, response_metadata={}, id='9ab843bc-58dc-4c62-a286-92d02e2c79c4'), HumanMessage(content='제주도행 항공편이었고, 작년 7월쯤이었어요. 예약 번호는 기억이 안 나는데, 항공권 번호는 123-4567890123 입니다. 항공편명은 대한항공 KE123편이었고, 제주에서 서울로 오는 편이었습니다.\n\n당시 항공편이 6시간 이상 지연되어서 예정된 일정을 모두 망쳤어요. 그래서 전액 환불을 요구합니다.', additional_kwargs={}, response_metadata={}, 